In [5]:
import requests
import json
import pandas as pd

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

In [ ]:
def query_dpe(params):
    url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

    response = requests.get(url="https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines", params=params)

    if response.status_code == 200:

        print("Requête réussie: ", response)
        result = response.json()
        result = result["results"]
        result = pd.DataFrame(result)
        return result

    else:
        print("Echec de la requête")
        return None

In [43]:

# Paramètres de la requête
params = {
    "page": 1,
    "size": 5,
    "select": "numero_dpe,date_reception_dpe,code_postal_ban,etiquette_dpe",
    "q": "69008",
    "q_fields": "code_postal_ban",
    "qs": "date_reception_dpe:[2023-06-29 TO 2023-08-30]"
}

# Effectuer la requête GET avec les paramètres
response = requests.get(base_url, params=params)

# Vérifier le statut de la réponse
print(f"Statut de la réponse : {response.status_code}")

# Si la requête a été effectuée avec succès, traiter le contenu
if response.status_code == 200:
    # Charger le contenu de la réponse en JSON
    content = response.json()
    
    # Afficher le nombre total de lignes dans la base de données
    print(f"Nombre total de lignes : {content['total']}")
    
    # Afficher les données récupérées
    df = content['results']
    
    # Afficher les dimensions du DataFrame (nombre de lignes et de colonnes)
    print(f"Dimensions des données récupérées : {len(df)}, {len(df[0]) if df else 0}")
    
    # Si nécessaire, afficher les données
    print(df)
else:
    print("Erreur lors de la requête")

Statut de la réponse : 200
Nombre total de lignes : 1616
Dimensions des données récupérées : 5, 5
[{'numero_dpe': '2369E2580120D', 'date_reception_dpe': '2023-07-31', 'code_postal_ban': '69008', 'etiquette_dpe': 'C', '_score': 25.891695}, {'numero_dpe': '2369E2580383G', 'date_reception_dpe': '2023-07-31', 'code_postal_ban': '69008', 'etiquette_dpe': 'C', '_score': 25.891695}, {'numero_dpe': '2369E2580402Z', 'date_reception_dpe': '2023-07-31', 'code_postal_ban': '69008', 'etiquette_dpe': 'C', '_score': 25.891695}, {'numero_dpe': '2369E2288153Q', 'date_reception_dpe': '2023-07-06', 'code_postal_ban': '69008', 'etiquette_dpe': 'E', '_score': 25.891695}, {'numero_dpe': '2369E2252038P', 'date_reception_dpe': '2023-07-04', 'code_postal_ban': '69008', 'etiquette_dpe': 'E', '_score': 25.891695}]


# Extraction des données complètes pour le Rhône

In [47]:
# Récupération des codes postaux
df = pd.read_csv("data/adresses-69.csv", sep=";")

C:\Users\axcano\AppData\Local\Temp\ipykernel_6952\2572907024.py:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/adresses-69.csv", sep=";")


In [60]:
# Récupérer les codes postaux uniques.
code_postaux = df["code_postal"].unique()

In [124]:
# Récupération le dataframe des logmeents en multipliant les requêtes.

df_list = list()

for code in code_postaux[:2]:

    params = {
        "page":1,
        "size": 500,
        "qs": f'code_postal_ban:"{code}" date_etablissement_dpe:2022'
    }

    content = query_dpe(params)

    print(content.shape)
    
    df_list.append(pd.DataFrame(content))

Requête réussie:  <Response [200]>
(500, 212)
Requête réussie:  <Response [200]>
(500, 208)


In [121]:
pd.concat(df_list)

,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,score_ban,conso_auxiliaires_ep,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,...,facteur_couverture_solaire_n1,description_generateur_chauffage_n2_installation_n2,usage_generateur_n2_installation_n2,type_generateur_n2_installation_n2,type_energie_generateur_n2_installation_n2,conso_chauffage_generateur_n2_installation_n2,provenance_id_rnb,id_rnb,surface_totale_capteurs_pv,nombre_module
0,Installation de chauffage simple,5012.4,Ballon électrique à accumulation vertical Autr...,0.47,1007.4,62.8,30.6,438.0,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Installation de chauffage simple,5046.4,Ballon électrique à accumulation vertical Autr...,0.34,1.0,41.7,28.2,0.0,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Installation de chauffage simple,1340.6,Ballon électrique à accumulation vertical Autr...,0.70,1.0,11.1,20.4,0.0,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Installation de chauffage simple,12891.6,Ballon électrique à accumulation vertical Caté...,0.98,33771.1,83.5,17.0,0.0,adresse non géocodée ban car aucune correspond...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Installation de chauffage simple,1102.6,Ballon électrique à accumulation vertical Caté...,0.69,406.0,8.7,21.9,176.5,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Installation de chauffage simple,3573.2,Chaudière gaz à condensation après 2015,0.96,741.3,12.9,28.1,322.3,adresse non géocodée ban car aucune correspond...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Reprise RNB,K6A8DH7GVAYQ,NaN,NaN
496,Installation de chauffage simple,2242.9,Chaudière gaz à condensation 2001-2015,0.71,1336.9,10.7,44.0,581.2,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Reprise RNB,19GZG2BTS7CB,NaN,NaN
497,Installation de chauffage simple,10921.0,Réseau de chaleur non isolé,0.71,2102.0,19.8,45.0,913.9,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Reprise RNB,1A7V5V5RN74N,NaN,NaN
498,Installation de chauffage avec insert ou poêle...,14129.3,Ballon électrique à accumulation vertical Caté...,0.38,1309.6,68.6,36.4,569.4,adresse géocodée ban à l'adresse,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Reprise RNB,92A7VA2DTKJE,NaN,NaN
